In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
data=pd.read_csv("../DataSet/Student_Profile.csv")
data

TOC  Data Science  Average Time Taken Preference of Learning  \
0    1             0                  30                    MCQ   
1    0             1                  45                  Essay   
2    1             1                  40                    MCQ   
3    0             0                  35                  Essay   
4    1             1                  50                    MCQ   

   Previous Score  Accuracy of Student  
0              85                 0.90  
1              78                 0.80  
2              92                 0.95  
3              80                 0.85  
4              88                 0.90

In [71]:
for i in data:
    print(data[i].dtype)

int64
int64
int64
object
int64
float64


In [72]:
data[data["Preference of Learning"].unique()]=pd.get_dummies(data["Preference of Learning"])*1
data=data.drop(columns=["Preference of Learning"])

minmax=StandardScaler()
data[["Previous Score","Average Time Taken"]]=minmax.fit_transform(data[["Previous Score", "Average Time Taken"]])
data

TOC  Data Science  Average Time Taken  Previous Score  Accuracy of Student  \
0    1             0           -1.414214        0.078087                 0.90   
1    0             1            0.707107       -1.288434                 0.80   
2    1             1            0.000000        1.444607                 0.95   
3    0             0           -0.707107       -0.897999                 0.85   
4    1             1            1.414214        0.663738                 0.90   

   MCQ  Essay  
0    0      1  
1    1      0  
2    0      1  
3    1      0  
4    0      1

# Question Set Preprocessing

In [2]:
question_data=pd.read_csv("../DataSet/Question_Set.csv")
question_data

TOC  Data Science  Expected Time to be solved Question Type  \
0      0             0                           5           MCQ   
1      1             1                          19           MCQ   
2      1             1                           9         Essay   
3      0             1                          42         Essay   
4      0             1                          31           MCQ   
..   ...           ...                         ...           ...   
495    0             0                          28         Essay   
496    1             0                          49           MCQ   
497    1             0                          56           MCQ   
498    0             1                          11         Essay   
499    1             1                          28         Essay   

    Question Difficulty  Accuracy of the question  
0                  Hard                      0.79  
1                  Easy                      0.70  
2                  Hard                      0.90  
3                  Easy                      0.89  
4                  Easy                      0.99  
..                  ...                       ...  
495                Hard                      0.68  
496                Easy                      0.56  
497                Hard                      0.84  
498                Easy                      0.60  
499              Medium                      0.84  

[500 rows x 6 columns]

In [3]:
question_data["Question Difficulty"].value_counts()

Question Difficulty
Hard      168
Medium    167
Easy      165
Name: count, dtype: int64

In [74]:
question_data[question_data["Question Type"].unique()]=pd.get_dummies(question_data["Question Type"])*1
question_data["Question Difficulty"]=question_data["Question Difficulty"].map({"Easy":1,"Medium":2,"Hard":3})
question_data[["Expected Time to be solved"]]=minmax.fit_transform(question_data[["Expected Time to be solved"]])
question_data=question_data.drop(columns=["Question Type"])
question_data

TOC  Data Science  Expected Time to be solved  Question Difficulty  \
0      0             0                   -1.530381                    3   
1      1             1                   -0.694368                    1   
2      1             1                   -1.291521                    3   
3      0             1                    0.679081                    1   
4      0             1                    0.022214                    1   
..   ...           ...                         ...                  ...   
495    0             0                   -0.156932                    3   
496    1             0                    1.097088                    1   
497    1             0                    1.515094                    3   
498    0             1                   -1.172090                    1   
499    1             1                   -0.156932                    2   

     Accuracy of the question  MCQ  Essay  
0                        0.79    0      1  
1                        0.70    0      1  
2                        0.90    1      0  
3                        0.89    1      0  
4                        0.99    0      1  
..                        ...  ...    ...  
495                      0.68    1      0  
496                      0.56    0      1  
497                      0.84    0      1  
498                      0.60    1      0  
499                      0.84    1      0  

[500 rows x 7 columns]

# Taking the Profile of one Student 

In [75]:
data.head(1)

TOC  Data Science  Average Time Taken  Previous Score  Accuracy of Student  \
0    1             0           -1.414214        0.078087                  0.9   

   MCQ  Essay  
0    0      1

# Implementing the Cosine Similarity

In [76]:
cosine_sim = cosine_similarity(question_data,data.head(1))
cosine_sim_df = pd.DataFrame(cosine_sim)
top_similar_students = cosine_sim_df[0].sort_values(ascending=False)
top_similar_students # Get the index of the top similar students

334    0.933007
127    0.913102
268    0.909222
272    0.902572
67     0.891638
         ...   
304   -0.278169
90    -0.280455
289   -0.281535
398   -0.291288
206   -0.294352
Name: 0, Length: 500, dtype: float64

In [77]:
data.columns = ["TOC", "Data Science", "Expected Time to be solved", "Question Difficulty", "Accuracy of the question", "MCQ", "Essay"]

In [78]:
data.head(1)

TOC  Data Science  Expected Time to be solved  Question Difficulty  \
0    1             0                   -1.414214             0.078087   

   Accuracy of the question  MCQ  Essay  
0                       0.9    0      1

In [79]:
# Renaming Collumsn
# data.columns = ["TOC", "Data Science", "Average Time Taken", "Previous Score", "Accuracy of Student", "MCQ", "Essay"]
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(n_neighbors=5, metric='cosine')
nn.fit(question_data)

cosine_sim = nn.kneighbors(data.head(1), 5, return_distance=False)
print(cosine_sim)

[[334 127 268 272  67]]
